In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time, pickle, os
from datetime import datetime, timedelta
import pandas as pd


In [2]:
USERNAME = "test0011test10101@gmail.com"
PASSWORD = "test11001tweetx"
COOKIES_FILE = "twitter_cookies.pkl"

In [5]:
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)


In [7]:
def login_and_save_cookies():
    print("🔐 تسجيل الدخول...")
    driver.get("https://x.com/login")
    time.sleep(5)
    driver.find_element(By.NAME, "text").send_keys(USERNAME + Keys.RETURN)
    time.sleep(3)
    driver.find_element(By.NAME, "password").send_keys(PASSWORD + Keys.RETURN)
    time.sleep(10)
    pickle.dump(driver.get_cookies(), open(COOKIES_FILE, "wb"))
    print("✅ تم تسجيل الدخول وحفظ الكوكيز")

In [9]:
def load_cookies_or_login():
    driver.get("https://x.com/")
    time.sleep(3)

    # لو في كوكيز محفوظة، نستخدمها
    if os.path.exists(COOKIES_FILE):
        cookies = pickle.load(open(COOKIES_FILE, "rb"))
        for cookie in cookies:
            driver.add_cookie(cookie)
        driver.get("https://x.com/")
        time.sleep(5)

        # ✅ تحقق إذا تم التحميل بنجاح أو تم تحويلك لصفحة login
        if "login" in driver.current_url.lower():
            print("⚠️ الكوكيز انتهت صلاحيتها، جاري تسجيل الدخول...")
            login_and_save_cookies()
        else:
            print("✅ تم تحميل الكوكيز والدخول للحساب")
    else:
        login_and_save_cookies()



In [11]:
def open_search_page():
    driver.get("https://x.com/home")
    time.sleep(5)

    search_url = "https://x.com/search?q=rakbank&src=typed_query&f=live"
    driver.get(search_url)
    time.sleep(8) 


In [13]:
# def scroll_page(pause=2, count=25):
#     for _ in range(count):
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(pause)


def scroll_until_end(pause_time=2, max_rounds=50):
    last_count = 0
    for i in range(max_rounds):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)

        tweets_now = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
        print(f"⬇️ الجولة {i+1} - عدد التغريدات: {len(tweets_now)}")

        if len(tweets_now) == last_count:
            print("✅ ما في تغريدات جديدة، تم الوصول للنهاية.")
            break
        last_count = len(tweets_now)


In [37]:
def scroll_until_no_more(pause=2, max_attempts=50):
    last_height = driver.execute_script("return document.body.scrollHeight")
    same_count = 0

    for i in range(max_attempts):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)

        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            same_count += 1
            print(f"⏳ لا تغيير بالجولة {i+1}... (محاولة {same_count})")
            if same_count >= 3:
                print("✅ تم الوصول لنهاية الصفحة.")
                break
        else:
            same_count = 0  # تم تحميل شيء جديد

        last_height = new_height


In [39]:
load_cookies_or_login()
open_search_page()
scroll_until_no_more(pause=2, max_attempts=60)


✅ تم تحميل الكوكيز والدخول للحساب
⏳ لا تغيير بالجولة 1... (محاولة 1)
⏳ لا تغيير بالجولة 2... (محاولة 2)
⏳ لا تغيير بالجولة 3... (محاولة 3)
✅ تم الوصول لنهاية الصفحة.


In [41]:
tweets = extract_tweets()

🧠 عدد التغريدات الظاهرة: 5


In [42]:
df = pd.DataFrame(tweets)
df

,Username,Date,Content,Likes,Retweets,Link
0,RAKBANK,2025-05-12 06:11,"Hello, we regret the inconvenience for the unp...",0,0,https://x.com/RAKBANKlive/status/1921810505968...
1,Customer Experience Live,2025-05-10 09:30,"We welcome Mohammad Ahteshamoddin, Senior VP &...",0,0,https://x.com/CXLiveTweets/status/192113567909...
2,وظائف اليوم,2025-05-10 07:45,وظائف بنك راس الخيمة RAKBANK للمواطنين وللوافدين,0,0,https://x.com/wazaifcom1/status/19211094330079...
3,Hariram,2025-05-08 11:15,"@RAKBANKlive\n : Hello RAKBANK team , I have a...",0,0,https://x.com/hariramtn/status/192043737735858...
4,T&momo,2025-05-08 11:09,使用準備が整っている銀行 仮想通貨xrp③\n\nRAKBANK\nアメリカン・エキスプレス...,0,0,https://x.com/momo_t111/status/192043589673990...


In [24]:
df.to_excel("rakbank_tweets_last2weeks.xlsx", index=False)
